## Steps for generating YoloNAS dlc
#### Note->Use python3.8 or above for generating onnx and python3.6 for generating dlc

In [ ]:
## Note- Use python3.8 or above for generating onnx

!pip install super-gradients==3.1.2

## Downloading Model from git repo
import torch
# Load model with pretrained weights
from super_gradients.training import models
from super_gradients.common.object_names import Models

model = models.get(Models.YOLO_NAS_S, pretrained_weights="coco")

# Prepare model for conversion
# Input size is in format of [Batch x Channels x Width x Height] where 640 is the standard COCO dataset dimensions
model.eval()
model.prep_model_for_conversion(input_size=[1, 3, 320, 320])

# Create dummy_input
dummy_input = torch.randn([1, 3, 320, 320], device="cpu")

# Convert model to onnx
torch.onnx.export(model, dummy_input, "yolo_nas_s.onnx", opset_version=11)

#### Enable python3.6 environment, to use SNPE SDK and then convert onnx to dlc

In [2]:
%%bash
snpe-onnx-to-dlc -i yolo_nas_s.onnx -o app/src/main/assets/yolo_nas_s.dlc

2023-11-24 11:09:04,341 - 235 - INFO - Successfully simplified the onnx model in child process
2023-11-24 11:09:04,494 - 235 - INFO - Successfully receive the simplified onnx model in main process
2023-11-24 11:09:04,587 - 235 - INFO - Successfully run shape inference in child process
2023-11-24 11:09:04,743 - 235 - INFO - Successfully receive the inferred model in main process
2023-11-24 11:09:05,526 - 235 - INFO - INFO_INITIALIZATION_SUCCESS: 
2023-11-24 11:09:05,665 - 235 - INFO - INFO_CONVERSION_SUCCESS: Conversion completed successfully
2023-11-24 11:09:05,714 - 235 - INFO - INFO_WRITE_SUCCESS: 


## Quantizing MobileNetSSD

In [ ]:
##STEPS to preprocess images

def preprocess(original_image):
    resized_image = cv2.resize(original_image, (320, 320))
    resized_image = resized_image/255
    return resized_image

import cv2
import numpy as np
import os

##Please download Coco2014 dataset and give the path here
dataset_path = "/local/mnt/workspace/shubgoya/exp_2.16/qidk-public-mirror/Solutions/VisionSolution1-ObjectDetection-YoloNas/val2017/"

!mkdir -p rawYoloNAS

filenames=[]
for path in os.listdir(dataset_path)[:5]:
    # check if current path is a file
    if os.path.isfile(os.path.join(dataset_path, path)):
        filenames.append(os.path.join(dataset_path, path))

for filename in filenames:
    original_image = cv2.imread(filename)
    img = preprocess(original_image)
    img = img.astype(np.float32)
    img.tofile("rawYoloNAS/"+filename.split("/")[-1].split(".")[0]+".raw")

In [19]:
%%bash
find rawYoloNAS -name *.raw > YoloInputlist.txt

In [20]:
%%bash
cat YoloInputlist.txt

rawYoloNAS/000000445834.raw
rawYoloNAS/000000404479.raw
rawYoloNAS/000000222991.raw
rawYoloNAS/000000555412.raw
rawYoloNAS/000000563267.raw


In [3]:
%%bash
snpe-dlc-quantize --input_dlc app/src/main/assets/yolo_nas_s.dlc --input_list YoloInputlist.txt --use_enhanced_quantizer --use_adjusted_weights_quantizer --axis_quant --output_dlc app/src/main/assets/Quant_intermediate_yoloNas_s_320.dlc


[INFO] InitializeStderr: DebugLog initialized.
[INFO] Processed command-line arguments
[INFO] Quantized parameters
[INFO] Generated activations
[INFO] Saved quantized dlc to: app/src/main/assets/Quant_intermediate_yoloNas_s_320.dlc
[INFO] DebugLog shutting down.


     0.1ms [  INFO ] Inferences will run in sync mode
     0.1ms [  INFO ] Initializing logging in the backend. Callback: [0xcf3a80], Log Level: [3]
     0.1ms [  INFO ] No BackendExtensions lib provided;initializing NetRunBackend Interface
     0.1ms [WARNING] Unable to find a device with NetRunDeviceKeyDefault in Library NetRunBackendLibKeyDefault
     0.2ms [  INFO ] Entering QuantizeRuntimeApp flow
   327.3ms [  INFO ] cleaning up resources for input tensors
   327.4ms [  INFO ] cleaning up resources for output tensors
   513.3ms [  INFO ] cleaning up resources for input tensors
   513.3ms [  INFO ] cleaning up resources for output tensors
   702.5ms [  INFO ] cleaning up resources for input tensors
   702.5ms [  INFO ] cleaning up resources for output tensors
   888.7ms [  INFO ] cleaning up resources for input tensors
   888.7ms [  INFO ] cleaning up resources for output tensors
  1066.1ms [  INFO ] cleaning up resources for input tensors
  1066.1ms [  INFO ] cleaning up resource

In [7]:
%%bash
snpe-dlc-graph-prepare --input_dlc app/src/main/assets/Quant_intermediate_yoloNas_s_320.dlc --set_output_tensors 885,877 --output_dlc app/src/main/assets/Quant_yoloNas_s_320.dlc --htp_socs sm8550

[INFO] InitializeStderr: DebugLog initialized.
[WARNING] Input[0] has Datatype 0x408.
[INFO] SNPE HTP Offline Prepare: Attempting to create cache for SM8550
[INFO] Attempting to open dynamically linked lib: libHtpPrepare.so
[INFO] dlopen libHtpPrepare.so SUCCESS handle 0x2205e30
[INFO] Found Interface Provider (v2.10)
[USER_WARNING] QnnDsp <W> Initializing HtpProvider
[USER_WARNING] QnnDsp <W> HTP arch will be deprecated, please set SoC id instead.
[USER_WARNING] QnnDsp <W> Cost Based unsupported on soc SM8550
[USER_INFO] Platform option not set
[USER_INFO] Offline Prepare VTCM size(MB) selected = 8
[USER_WARNING] QnnDsp <W> Output padding param cannot be set explicitly. Skipping param
[USER_WARNING] QnnDsp <W> Output padding param cannot be set explicitly. Skipping param



====== Prepared Graph's Summary ======
Spill fill buffer size = 0



[USER_INFO] Backend Mgr ~Dtor called for backend HTP
[USER_INFO] Cleaning up Context handle:0x1
[USER_INFO] BackendTerminate triggered
[USER_WARNING] QnnDsp <W> Device 0x1 not found in registry
[INFO] SNPE HTP Offline Prepare: Successfully created cache for SM8550
[INFO] SNPE HTP Offline Prepare: Saved cached DLC to app/src/main/assets/Quant_yoloNas_s_320.dlc
[USER_INFO] BackendTerminate triggered
[INFO] DebugLog shutting down.
